In [ ]:
# loading libraries for machine learning - predection analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# loading libraries for machine learning - predection analysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

import os
print(os.listdir())

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# loading the file
hotel_df = pd.read_csv("/Users/georgekalfas/Downloads/ML-HotelReservations/EDA-ML/Hotel Reservations.csv")

# Prediction Models

## Logistic Regression Model for Predicting Reservation Cancellation

### Method Overview
Logistic Regression is a statistical method used for predicting categorical outcomes. In the context of hotel reservation cancellations, it's applied to predict whether a customer will cancel their reservation or not.

### Steps Involved:
1. **Data Preparation**:
   - **Feature Selection**: Identifying predictive features and the target variable (`booking_status`).
   - **Encoding Categorical Variables**: Using label encoding to convert categorical variables into numerical format.
   - **Splitting the Dataset**: Separating the dataset into training and testing sets to evaluate the model's performance.

2. **Feature Standardization**:
   - **Scaling Features**: Standardizing the features using `StandardScaler` to ensure uniformity in data distribution.

3. **Logistic Regression Model**:
   - **Model Creation**: Initializing and training the Logistic Regression model using the training dataset.

4. **Model Evaluation**:
   - **Predictions**: Making predictions on the test dataset.
   - **Evaluation Metrics**: Assessing the model's performance using metrics like accuracy score, classification report, and confusion matrix.

### Results:
- **Accuracy Score**: The accuracy score obtained using Logistic Regression is displayed, indicating the model's overall predictive accuracy.
- **Classification Report**: Provides detailed information on precision, recall, and F1-score for each class ('Canceled' and 'Not Canceled').
- **Confusion Matrix**: Visual representation of predicted versus actual values to understand the model's performance.

### Conclusion:
Logistic Regression serves as an initial predictive model. The accuracy score and evaluation metrics help to understand its predictive capability in determining reservation cancellations.


In [ ]:
# selecting the target column for the predictions
target_column = 'booking_status'

# selecting features and target variable
features = hotel_df.drop(target_column, axis=1) 
target = hotel_df[target_column]

# handling categorical columns
label_encoder = LabelEncoder()
for column in features.columns:
    if features[column].dtype == 'object':
        features[column] = label_encoder.fit_transform(features[column])

# splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# standardizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# creating and train the Logistic Regression model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# making the predictions
predictions = model.predict(X_test)

# Evaluating the model
lr_accuracy = accuracy_score(y_test, predictions) * 100
print(f"The accuracy score achieved using Logistic Regression is: {lr_accuracy:.2f}%")

print("\nClassification Report for Logistic Regression:")
print(classification_report(y_test, predictions))

print("\nConfusion Matrix for Logistic Regression:")
print(confusion_matrix(y_test, predictions))

In [ ]:
# creating a function for the confusion matrix
def plot_confusion_matrix(conf_matrix, labels):
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, predictions)

# Define the class labels ('Not Canceled', 'Canceled')
class_labels = ['Not Canceled', 'Canceled']

# Plot the confusion matrix using the function
plot_confusion_matrix(conf_matrix, class_labels)

## Naive Bayes Model for Predicting Reservation Cancellation

### Method Overview
Naive Bayes is a probabilistic machine learning method that uses Bayes' theorem to make predictions. In the context of hotel reservation cancellations, the Gaussian Naive Bayes classifier is employed to predict whether a customer will cancel their reservation or not.

### Steps Involved:
1. **Model Creation**:
   - **Model Selection**: Using the Gaussian Naive Bayes model for training.
   - **Training the Model**: Fitting the model using the training dataset.

2. **Predictions and Evaluation**:
   - **Making Predictions**: Using the trained model to make predictions on the test dataset.
   - **Evaluation Metrics**: Assessing the model's performance using metrics like accuracy score, classification report, and confusion matrix.

### Results:
- **Accuracy Score**: The accuracy score obtained using the Naive Bayes model is displayed, representing the overall predictive accuracy.
- **Classification Report**: Detailed metrics such as precision, recall, and F1-score for each class ('Canceled' and 'Not Canceled').
- **Confusion Matrix**: Visualization of predicted versus actual values to assess the model's performance.

### Conclusion:
Naive Bayes, specifically the Gaussian variant, is applied for predictive analysis. The accuracy score and evaluation metrics provide insights into its effectiveness in predicting reservation cancellations.


In [ ]:
# Creating and training the Naive Bayes (Gaussian) model
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Making predictions using the trained model
nb_predictions = nb_model.predict(X_test)

# Evaluating the Naive Bayes model
nb_accuracy = accuracy_score(y_test, nb_predictions) * 100
print(f"The accuracy score achieved using Naive Bayes is: {nb_accuracy:.2f}%")

print("\nClassification Report for Naive Bayes:")
print(classification_report(y_test, nb_predictions))

print("\nConfusion Matrix for Naive Bayes:")
print(confusion_matrix(y_test, nb_predictions))


In [ ]:
# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, nb_predictions)

# Define the class labels ('Not Canceled', 'Canceled')
class_labels = ['Not Canceled', 'Canceled']

# Plot the confusion matrix using the function
plot_confusion_matrix(conf_matrix, class_labels)

## Support Vector Machine (SVM) for Reservation Cancellation Prediction

### Method Overview
Support Vector Machine is a powerful supervised learning algorithm used for classification tasks. In this analysis, SVM with various kernel functions (Linear, Polynomial, RBF) is applied to predict hotel reservation cancellations.

### Steps Involved:
1. **Linear Kernel SVM**:
   - Model Creation: Using the SVM model with a linear kernel.
   - Model Training: Fitting the SVM model using the training dataset.
   - Predictions: Making predictions and evaluating the model using accuracy, classification report, and confusion matrix.

2. **Polynomial Kernel SVM**:
   - Model Creation: Employing the SVM model with a polynomial kernel (degree=3).
   - Model Training: Fitting the SVM model to the training data.
   - Predictions and Evaluation: Assessing accuracy and classification metrics.

3. **RBF Kernel SVM**:
   - Model Creation: Utilizing the SVM model with a radial basis function (RBF) kernel.
   - Model Training: Fitting the SVM model with the RBF kernel to the training dataset.
   - Predictions and Evaluation: Evaluating accuracy and classification performance.

### Results:
- **Accuracy Scores**: Displaying the accuracy achieved by SVM models with different kernels (Linear, Polynomial, RBF).
- **Classification Reports**: Detailed metrics such as precision, recall, and F1-score for each class ('Canceled' and 'Not Canceled').
- **Confusion Matrices**: Visualization of predicted versus actual values for assessing model performance.

### Conclusion:
SVM, with its various kernel functions, is employed for reservation cancellation prediction. The evaluation metrics aid in understanding the effectiveness of SVM with different kernels in this predictive task.


In [ ]:
# Linear kernel
svm_linear = SVC(kernel='linear', random_state=42)
svm_linear.fit(X_train, y_train)
svm_linear_predictions = svm_linear.predict(X_test)

svm_linear_accuracy = accuracy_score(y_test, svm_linear_predictions) * 100
print(f"The accuracy score achieved using SVM with Linear Kernel is: {svm_linear_accuracy:.2f}%")

print("\nClassification Report for SVM with Linear Kernel:")
print(classification_report(y_test, svm_linear_predictions))

print("\nConfusion Matrix for SVM with Linear Kernel:")
print(confusion_matrix(y_test, svm_linear_predictions))

# Polynomial kernel
svm_poly = SVC(kernel='poly', degree=3, random_state=42)
svm_poly.fit(X_train, y_train)
svm_poly_predictions = svm_poly.predict(X_test)

svm_poly_accuracy = accuracy_score(y_test, svm_poly_predictions) * 100
print(f"\nThe accuracy score achieved using SVM with Polynomial Kernel is: {svm_poly_accuracy:.2f}%")

print("\nClassification Report for SVM with Polynomial Kernel:")
print(classification_report(y_test, svm_poly_predictions))

print("\nConfusion Matrix for SVM with Polynomial Kernel:")
print(confusion_matrix(y_test, svm_poly_predictions))

# RBF kernel
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train, y_train)
svm_rbf_predictions = svm_rbf.predict(X_test)

svm_rbf_accuracy = accuracy_score(y_test, svm_rbf_predictions) * 100
print(f"\nThe accuracy score achieved using SVM with RBF Kernel is: {svm_rbf_accuracy:.2f}%")

print("\nClassification Report for SVM with RBF Kernel:")
print(classification_report(y_test, svm_rbf_predictions))

print("\nConfusion Matrix for SVM with RBF Kernel:")
print(confusion_matrix(y_test, svm_rbf_predictions))

In [ ]:
# defining labels for confusion matrices
labels = ['Not Canceled', 'Canceled']

# Calculate confusion matrices
conf_matrix_linear = confusion_matrix(y_test, svm_linear_predictions)
conf_matrix_poly = confusion_matrix(y_test, svm_poly_predictions)
conf_matrix_rbf = confusion_matrix(y_test, svm_rbf_predictions)

# creating figure and axes
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# plotting confusion matrix for Linear Kernel
sns.heatmap(conf_matrix_linear, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels, ax=axs[0])
axs[0].set_title('SVM with Linear Kernel')

# plotting confusion matrix for Polynomial Kernel
sns.heatmap(conf_matrix_poly, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels, ax=axs[1])
axs[1].set_title('SVM with Polynomial Kernel')

# plotting confusion matrix for RBF Kernel
sns.heatmap(conf_matrix_rbf, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels, ax=axs[2])
axs[2].set_title('SVM with RBF Kernel')

# displaying the figure
plt.tight_layout()
plt.show()

## K Nearest Neighbors (KNN) for Reservation Cancellation Prediction

### Method Overview
K Nearest Neighbors is a versatile algorithm used for classification tasks based on feature similarity. In this analysis, KNN with 5 neighbors is applied to predict hotel reservation cancellations.

### Steps Involved:
1. **Model Creation and Training**:
   - Creating KNN Model: Instantiating the K Nearest Neighbors model with 5 neighbors.
   - Model Training: Training the KNN model using the provided training dataset.

2. **Predictions and Evaluation**:
   - Making Predictions: Generating predictions on the test dataset.
   - Model Evaluation: Assessing model accuracy and classification metrics such as precision, recall, and F1-score.

### Results:
- **Accuracy Score**: Displaying the achieved accuracy by the KNN model.
- **Classification Report**: Detailed classification metrics providing insights into model performance.
- **Confusion Matrix**: Visualization depicting predicted versus actual values for evaluating model effectiveness.

### Conclusion:
K Nearest Neighbors, with its feature similarity-based classification approach, is employed for predicting reservation cancellations. The accuracy score and classification metrics help understand the model's predictive performance.


In [ ]:
# creating and training the K Nearest Neighbors (KNN) model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# making predictions using the trained KNN model
knn_predictions = knn_model.predict(X_test)

# evaluating the KNN model
knn_accuracy = accuracy_score(y_test, knn_predictions) * 100
print(f"The accuracy score achieved using K Nearest Neighbors (KNN) is: {knn_accuracy:.2f}%")

print("\nClassification Report for K Nearest Neighbors (KNN):")
print(classification_report(y_test, knn_predictions))

print("\nConfusion Matrix for K Nearest Neighbors (KNN):")
print(confusion_matrix(y_test, knn_predictions))

In [ ]:
# calculating the confusion matrix
conf_matrix = confusion_matrix(y_test, knn_predictions)

# defining the class labels ('Not Canceled', 'Canceled')
class_labels = ['Not Canceled', 'Canceled']

# plotting the confusion matrix using the function
plot_confusion_matrix(conf_matrix, class_labels)

## Decision Tree Classifier for Reservation Cancellation Prediction

### Method Overview
Decision Tree Classifier is a tree-like model that makes decisions by splitting data based on feature values. In this analysis, a Decision Tree model is utilized to predict hotel reservation cancellations.

### Steps Involved:
1. **Hyperparameter Tuning**:
   - Finding Best Parameters: Looping through various random states to determine the one that yields the highest accuracy.
   - Model Instantiation: Creating a Decision Tree model with the best random state found.

2. **Model Training and Prediction**:
   - Model Training: Fitting the Decision Tree model using the training dataset.
   - Generating Predictions: Using the trained model to predict reservation cancellations on the test dataset.

3. **Evaluation**:
   - Confusion Matrix: Computing the confusion matrix to analyze predicted versus actual results.

### Results:
- **Max Accuracy Achieved**: Displaying the highest accuracy attained by the Decision Tree model.
- **Best Random State**: Identifying the random state that produced the most accurate predictions.
- **Confusion Matrix**: Visualization illustrating the model's performance in predicting cancellation outcomes.

### Conclusion:
Decision Tree Classifier, through its branching structure based on features, is employed for predicting reservation cancellations. The analysis provides insights into model accuracy and its effectiveness in making cancellation predictions.


In [ ]:
dt_max_accuracy = 0
best_x = 0

for x in range(200):
    dt = DecisionTreeClassifier(random_state=x)
    dt.fit(X_train, y_train)
    Y_pred_dt = dt.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_dt, y_test) * 100, 2)
    if current_accuracy > dt_max_accuracy:
        dt_max_accuracy = current_accuracy
        best_x = x

print(f"Max accuracy achieved: {dt_max_accuracy}%")
print(f"Best random state: {best_x}")

dt = DecisionTreeClassifier(random_state=best_x)
dt.fit(X_train, y_train)
Y_pred_dt = dt.predict(X_test)

# computing confusion matrix
cm = confusion_matrix(y_test, Y_pred_dt)
print("\nConfusion Matrix for Decision Tree Classifier:")
print(cm)

# plotting confusion matrix
plot_confusion_matrix(cm, ['Not Canceled', 'Canceled'])

## Random Forest Classifier for Reservation Cancellation Prediction

### Method Overview
Random Forest Classifier is an ensemble learning technique that constructs multiple decision trees and merges their predictions to improve accuracy. In this analysis, Random Forest is employed for predicting hotel reservation cancellations.

### Steps Involved:
1. **Hyperparameter Tuning**:
   - Finding Best Parameters: Iterating through different random states to identify the one yielding the highest accuracy.
   - Model Initialization: Creating a Random Forest model with the best random state found.

2. **Model Training and Prediction**:
   - Model Fitting: Training the Random Forest model using the training dataset.
   - Prediction Generation: Utilizing the trained model to predict reservation cancellations on the test dataset.

3. **Evaluation**:
   - Confusion Matrix: Computation of the confusion matrix to evaluate the model's predictive performance.

### Results:
- **Max Accuracy Achieved**: Displaying the highest accuracy attained by the Random Forest model.
- **Best Random State**: Identifying the random state that produced the most accurate predictions.
- **Confusion Matrix**: Visualization illustrating the model's performance in predicting cancellation outcomes.

### Conclusion:
Random Forest Classifier, by aggregating multiple decision trees, is used to predict hotel reservation cancellations. The analysis provides insights into model accuracy and its effectiveness in making cancellation predictions.


In [ ]:
rf_max_accuracy = 0
best_x = 0

for x in range(200):
    rf = RandomForestClassifier(random_state=x)
    rf.fit(X_train, y_train)
    Y_pred_rf = rf.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_rf, y_test) * 100, 2)
    if current_accuracy > rf_max_accuracy:
        rf_max_accuracy = current_accuracy
        best_x = x

print(f"Max accuracy achieved: {rf_max_accuracy}%")
print(f"Best random state: {best_x}")

rf = RandomForestClassifier(random_state=best_x)
rf.fit(X_train, y_train)
Y_pred_rf = rf.predict(X_test)

cm = confusion_matrix(y_test, Y_pred_rf)
print("\nConfusion Matrix for Random Forest Classifier:")
print(cm)

plot_confusion_matrix(cm, ['Not Canceled', 'Canceled'])

## XGBoost Classifier for Reservation Cancellation Prediction

### Method Overview
XGBoost (Extreme Gradient Boosting) Classifier is an ensemble learning method that enhances the accuracy of prediction models. It iteratively builds multiple decision trees and combines their outputs. This analysis employs XGBoost for predicting hotel reservation cancellations.

### Steps Involved:
1. **Data Encoding**:
   - Label Encoding: Converting categorical target classes into numeric labels for training and test datasets.

2. **Model Training and Prediction**:
   - Model Fitting: Training the XGBoost Classifier using the encoded training dataset.
   - Prediction Generation: Using the trained model to predict reservation cancellations on the encoded test dataset.

3. **Evaluation**:
   - Accuracy Computation: Calculating the accuracy score to assess the model's performance.
   - Confusion Matrix: Generating the confusion matrix to evaluate the classification results.

### Results:
- **Accuracy Score**: Displaying the accuracy achieved by the XGBoost Classifier in predicting reservation cancellations.
- **Confusion Matrix**: Visualization depicting the model's predictive performance through classification results.

### Conclusion:
XGBoost Classifier, an advanced boosting algorithm, is utilized to predict hotel reservation cancellations. The analysis showcases the accuracy achieved and the model's effectiveness in making accurate cancellation predictions.


In [ ]:
# converting target classes to numeric labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# defining and fit the XGBoost classifier
xgb_clf = xgb.XGBClassifier(random_state=42)
xgb_clf.fit(X_train, y_train_encoded)

# making predictions using the XGBoost model
y_pred_xgb = xgb_clf.predict(X_test)

# calculating accuracy
accuracy_xgb = accuracy_score(y_test_encoded, y_pred_xgb) * 100
print(f"The accuracy score achieved using XGBoost Classifier is: {accuracy_xgb:.2f}%")

# computing confusion matrix
cm_xgb = confusion_matrix(y_test_encoded, y_pred_xgb)
print("\nConfusion Matrix for XGBoost Classifier:")
print(cm_xgb)

# plotting confusion matrix
plot_confusion_matrix(cm_xgb, ['Not_Canceled', 'Canceled'])

### Comparison of Accuracy Scores from Prediction Models

The following bar chart presents a visual comparison of the accuracy scores obtained from various prediction models used to predict hotel reservation cancellations. Each bar represents the accuracy score achieved by a specific prediction model. The accuracy score indicates the model's performance in correctly predicting reservation cancellations.

The models evaluated include Logistic Regression, Naive Bayes, SVM with different kernels (Linear, Polynomial, and RBF), K Nearest Neighbors (KNN), Decision Tree, Random Forest, and XGBoost. Higher accuracy scores suggest better predictive performance.

In [ ]:
# Accuracy scores obtained from different prediction models
model_names = ['Logistic Regression', 'Naive Bayes', 'SVM Linear', 'SVM Polynomial', 'SVM RBF', 'KNN', 'Decision Tree', 'Random Forest', 'XGBoost']
accuracy_scores = [lr_accuracy, nb_accuracy, svm_linear_accuracy, svm_poly_accuracy, svm_rbf_accuracy, knn_accuracy, dt_max_accuracy, rf_max_accuracy, accuracy_xgb]

# Creating bar charts for accuracy scores
plt.figure(figsize=(10, 6))
plt.bar(model_names, accuracy_scores)
plt.xlabel('Prediction Models')
plt.ylabel('Accuracy Score')
plt.title('Accuracy Scores of Prediction Models')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Summary of Analysis

### Logistic Regression:
- Utilized logistic regression to predict hotel reservation cancellations with an accuracy of 81%.
- Processed categorical data through label encoding and standardized features using StandardScaler.
- Generated classification report and confusion matrix to evaluate model performance.

### Naive Bayes:
- Employed Gaussian Naive Bayes resulting in an accuracy of 44% for predicting reservation cancellations.
- Generated classification report and confusion matrix to assess model predictions.

### Support Vector Machine (SVM):
- Explored SVM with linear, polynomial, and RBF kernels achieving accuracy scores ranging from 80% to 84%.
- Evaluated classification reports and confusion matrices for each kernel to analyze performance.

### K Nearest Neighbors (KNN):
- Used KNN with an accuracy score of 84% in predicting reservation cancellations.
- Assessed model performance using classification report and confusion matrix.

### Decision Tree, Random Forest, and XGBoost:
- Implemented Decision Tree, Random Forest, and XGBoost models achieving accuracy scores of 85.11%, 90.07%, and 89%, respectively.
- Conducted evaluations through confusion matrices and classification reports to analyze model predictions.

The analysis encompassed various machine learning algorithms to predict reservation cancellations in hotels, each demonstrating distinct accuracy levels. To enhance these models and potentially improve predictive performance, several strategies are recommended.

## Strategies for Enhancing Prediction Models

### Logistic Regression:
- **Feature Engineering:** Explore interaction terms between variables, create dummy variables for categorical data, or try polynomial features for numeric attributes.
- **Regularization Techniques:** Implement regularization techniques like L1 (Lasso) or L2 (Ridge) regularization to prevent overfitting.

### Naive Bayes:
- **Feature Selection:** Identify important features that significantly impact predictions. Experiment with different feature sets to observe their influence on performance.

### Support Vector Machine (SVM):
- **Hyperparameter Tuning:** Fine-tune hyperparameters such as C, gamma, and kernel coefficients to improve SVM performance further.
- **Class Imbalance Handling:** Implement techniques like oversampling, undersampling, or using class weights to handle imbalanced classes.

### K Nearest Neighbors (KNN):
- **Feature Scaling:** Experiment with different scaling techniques to normalize or standardize features for KNN.
- **Optimal 'k' Value:** Evaluate various values of 'k' to determine the optimal number of neighbors for classification.

### Decision Tree:
- **Pruning Techniques:** Apply pruning techniques like setting the maximum depth or minimum samples per leaf to avoid overfitting and improve generalization.

### Random Forest:
- **Ensemble Methods:** Experiment with boosting or bagging techniques by adjusting parameters like the number of trees or max depth to enhance predictive accuracy.
- **Feature Importance:** Explore feature importance scores to understand which attributes strongly impact predictions.

### XGBoost:
- **Parameter Tuning:** Optimize hyperparameters like learning rate, tree depth, and number of estimators to enhance model performance.
- **Regularization:** Implement regularization techniques such as L1 and L2 regularization to control overfitting.

### General Enhancements:
- **Feature Importance Analysis:** Investigate which features contribute the most to the models. Drop less important or redundant features.
- **Address Class Imbalance:** Utilize techniques like oversampling, undersampling, or Synthetic Minority Over-sampling Technique (SMOTE) to handle imbalanced classes.
- **Cross-Validation:** Implement cross-validation techniques to ensure robustness and validate model performance across different subsets of data.

By focusing on feature engineering, hyperparameter tuning, handling class imbalance, and exploring ensemble methods, it can potentially improve the models' accuracy and robustness in predicting hotel reservation cancellations.